In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

import analysis as a

p = make_paths(a)

In [ ]:
nmf_map = ccal.load_nmf_map(p['nmf_map_file_path'])

nmf_map.plot_nmf_map(
    'h',
    element_marker_size=a.NMF_MAP_H_ELEMENT_MARKER_SIZE,
)

In [ ]:
w = pd.DataFrame(
    nmf_map.wt.T,
    index=nmf_map.wt_elements,
    columns=nmf_map.nodes,
)

In [ ]:
feature_x_sample = pd.read_table(
    p['feature_x_sample_file_path'],
    index_col=0,
)

feature_1d_context_matrix = pd.read_table(
    p['feature_1d_context_matrix_file_path'],
    index_col=0,
)

sample_1d_context_matrix = pd.read_table(
    p['sample_1d_context_matrix_file_path'],
    index_col=0,
).T

signal_matrix = pd.read_table(
    p['signal_matrix_file_path'],
    index_col=0,
)

In [ ]:
for name, feature_x_sample_ in (
    (
        'Feature 1D Context Matrix',
        feature_1d_context_matrix,
    ),
    (
        'Sample 1D Context Matrix',
        sample_1d_context_matrix,
    ),
    (
        'Signal Matrix',
        signal_matrix,
    ),
):

    if name == 'Signal Matrix':

        signal_matrix_ = signal_matrix

    else:

        negative_feature_x_sample = -feature_x_sample_.clip(upper=0)

        negative_feature_x_sample.index = feature_x_sample_.index.map(lambda str_: '(-) {}'.format(str_))

        positive_feature_x_sample = feature_x_sample_.clip(lower=0)

        positive_feature_x_sample.index = feature_x_sample_.index.map(lambda str_: '(+) {}'.format(str_))

        w_negative_features = w.index[w.index.str.startswith('(-)')]

        common_negative_features = w_negative_features & negative_feature_x_sample.index

        print('(-) common features: {}/{} ({:.2f}%)'.format(
            common_negative_features.size,
            w_negative_features.size,
            100 * common_negative_features.size / w_negative_features.size,
        ))

        w_positive_features = w.index[w.index.str.startswith('(+)')]

        common_positive_features = w_positive_features & positive_feature_x_sample.index

        print('(+) common features: {}/{} ({:.2f}%)'.format(
            common_positive_features.size,
            w_positive_features.size,
            100 * common_positive_features.size / w_positive_features.size,
        ))

        signal_matrix_ = pd.concat((
            negative_feature_x_sample.loc[common_negative_features],
            positive_feature_x_sample.loc[common_positive_features],
        ))

    h_solved = ccal.solve_for_nmf_h(
        signal_matrix_,
        w.loc[signal_matrix_.index],
    )

    predicted_element_states = nmf_map.predict(
        'h',
        h_solved,
        title=name,
        element_marker_size=a.NMF_MAP_H_ELEMENT_MARKER_SIZE,
    )

    ccal.plot_heat_map(
        h_solved,
        normalization_axis=0,
        normalization_method='-0-',
        column_annotation=predicted_element_states,
        cluster_axis=1,
        title=name,
        xaxis_title=h_solved.columns.name,
        yaxis_title=h_solved.index.name,
    )